In [1]:
#!pip install openai python-dotenv

In [2]:
#Libraries
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

import time
from dataclasses import dataclass
from typing import List, Dict, Optional, Callable

import pandas as pd

from collections import defaultdict

import re

# Parameters

In [3]:
MODELS_OPTIONS = {
    #GPT
    1:"gpt-5-chat",
    2:"gpt-5.2-chat",
    3:"gpt-5-nano",

    #o4
    4:"o4-mini",

    #DeepSeek
    5:"DeepSeek-V3.1"
}


In [4]:
# Select models to use
# Uncomment the line below to enable user input
#MODELS_selected = input(f"Select models to use (comma separated) from the following options:\n{MODELS_OPTIONS}\n")
MODELS_selected='1,2,3,4,5'
MODELS = [MODELS_OPTIONS[int(i)] for i in MODELS_selected.split(",")]
MODELS

['gpt-5-chat', 'gpt-5.2-chat', 'gpt-5-nano', 'o4-mini', 'DeepSeek-V3.1']

In [5]:
# Load task instructions
tasks_df = pd.read_csv("simple_tasks.csv")
tasks_df



,id,task_type,input_text,expected
0,food_001,extract_food_items,"The food was not good, but I liked how the wai...",hamburger
1,food_002,extract_food_items,"I came for coffee, stayed for the croissant. T...",coffee; croissant
2,food_003,extract_food_items,We didn't order dessert. The pizza was great a...,pizza; salad
3,food_004,extract_food_items,"The sushi was overpriced; however, the miso so...",sushi; miso soup
4,food_005,extract_food_items,"I asked for fries, but they brought onion ring...",fries; onion rings; burger
...,...,...,...,...
61,contr_006,detect_contradiction,I don't know his name. I think his name is Car...,maybe
62,contr_007,detect_contradiction,We shipped the order yesterday; today we reali...,yes
63,contr_008,detect_contradiction,The meeting is at 10am. Please join at 10:00 i...,no
64,contr_009,detect_contradiction,"I can’t recommend Vendor A over Vendor B, but ...",no


In [6]:
SYSTEM_SIMPLE = (
    "You are an information extraction system. "
    "Follow the instruction exactly. "
    "Answer in English. "
    "Do not add explanations."
)

TASK_INSTRUCTIONS = {
    "extract_food_items":
        "Extract ALL food/drink items mentioned in the text. "
        "Do NOT infer, filter, or judge; just extract items that are mentioned. "
        "Return a semicolon-separated list in the order they appear. "
        "Return an empty string if none.",
    "extract_last_name":
        "Extract the last name(s) from the full name. "
        "Return ONLY the last name(s) as they appear (including particles like 'de', 'del', 'bin' when present). "
        "Return a single string.",
    "extract_legal_entities":
        "Extract the legal entity names mentioned in the text. "
        "Preserve the entity names as written. "
        "Return a semicolon-separated list. "
        "If none, return an empty string.",
    "extract_decision":
        "Extract the final decision from the text. "
        "Return ONLY one of: approved, denied, pending. "
        "If unclear, choose pending.",
    "extract_primary_date_iso":
        "Extract the PRIMARY date mentioned (the first main date / the main deadline / the main event date). "
        "Return ONLY the date in ISO format YYYY-MM-DD. "
        "If no date, return an empty string.",
    "detect_contradiction":
        "Does the text contain a contradiction? "
        "Return ONLY one of: yes, no, maybe. "
        "Use 'maybe' only if it's genuinely ambiguous.",
}


In [7]:
# Adddiotal tasks
tasks_df["instruction"] = tasks_df["task_type"].map(TASK_INSTRUCTIONS)
tasks_df["input"] = tasks_df["input_text"]

#tasks_df = tasks_df[["id", "instruction", "input", "expected"]]

tasks_df.head()

,id,task_type,input_text,expected,instruction,input
0,food_001,extract_food_items,"The food was not good, but I liked how the wai...",hamburger,Extract ALL food/drink items mentioned in the ...,"The food was not good, but I liked how the wai..."
1,food_002,extract_food_items,"I came for coffee, stayed for the croissant. T...",coffee; croissant,Extract ALL food/drink items mentioned in the ...,"I came for coffee, stayed for the croissant. T..."
2,food_003,extract_food_items,We didn't order dessert. The pizza was great a...,pizza; salad,Extract ALL food/drink items mentioned in the ...,We didn't order dessert. The pizza was great a...
3,food_004,extract_food_items,"The sushi was overpriced; however, the miso so...",sushi; miso soup,Extract ALL food/drink items mentioned in the ...,"The sushi was overpriced; however, the miso so..."
4,food_005,extract_food_items,"I asked for fries, but they brought onion ring...",fries; onion rings; burger,Extract ALL food/drink items mentioned in the ...,"I asked for fries, but they brought onion ring..."


# Configuration

In [8]:
# 1. Load environment variables from .env
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

print("Endpoint loaded:", endpoint)

Endpoint loaded: https://rafae-mj2b6wcp-eastus2.cognitiveservices.azure.com


In [9]:
# 2. Create AzureOpenAI client
client = AzureOpenAI(
    api_version="2024-12-01-preview",          # it is in the snippet
    azure_endpoint=endpoint,
    api_key=api_key,
)

In [10]:
# 2. Test call to chat completions
response = client.chat.completions.create(
    model=MODELS[1], 
    messages=[
        {"role": "system", 
         "content": "You are a helpful assistant that answers in Spanish."},
        {"role": "user",
         "content": "Hola, ¿me puedes confirmar que ya estoy usando Azure OpenAI?"},
    ],
)

print("\nRespuesta del modelo:\n")
print(response.choices[0].message.content)


Respuesta del modelo:

Hola 🙂  
No puedo confirmarlo directamente, porque no tengo visibilidad sobre la infraestructura o el servicio desde el que te estás conectando. Es decir, desde aquí no puedo saber si tu aplicación o entorno está usando **Azure OpenAI** u otro proveedor.

Lo que sí puedo hacer es decirte **cómo verificarlo tú mismo**:

### Cómo saber si estás usando Azure OpenAI
1. **Revisa el endpoint**  
   - Azure OpenAI usa URLs como:  
     ```
     https://<nombre-recurso>.openai.azure.com/
     ```
   - Si ves `openai.azure.com`, estás en Azure OpenAI.

2. **Clave de API**
   - En Azure OpenAI, la clave se obtiene desde el **Portal de Azure**, dentro del recurso *Azure OpenAI*.
   - No es la misma clave que la de OpenAI “directo” (openai.com).

3. **Configuración del SDK**
   - En Azure normalmente se configuran variables como:
     - `AZURE_OPENAI_ENDPOINT`
     - `AZURE_OPENAI_API_KEY`
     - `AZURE_OPENAI_DEPLOYMENT_NAME`

4. **Portal de Azure**
   - Si puedes ver y ad

In [11]:
# Pricing for 1M tokens (us east 2)
# Reference: https://portal.azure.com/#view/Microsoft_Azure_CostManagement/Menu/~/costanalysis/scope/%2Fsubscriptions%2Fe1edd698-277f-49c0-ac32-a849931aac08/open/costanalysisv3.resources/openedBy/Subscription.CostAnalysis.CBR%3AResources
# Others data: https://azure.microsoft.com/es-mx/pricing/details/cognitive-services/openai-service/#pricing 
# Consider if we need to include cost per use or something similar additionally to the token pricing
MODEL_PRICING = {
    "gpt-5-chat": {
        "input_1M": 0.05,   # USD por 1M input tokens
        "output_1M": 0.08,  # USD por 1M output tokens
    },
    "o4-mini": {
        "input_1M": 0.04,
        "output_1M": 0.04,
    },
    "DeepSeek-V3.1": {
        "input_1M": 0.04,
        "output_1M": 0.05,
    },
    "gpt-5.2-chat": {
        "input_1M": 0.07,
        "output_1M": 0.08,
    },
    "gpt-5-nano": {
        "input_1M": 0.009, # <$0.01 USD per 1K input tokens
        "output_1M": 0.01,
    },
}

def per_token_prices(model_name: str):
    p = MODEL_PRICING.get(model_name, {})
    in_per_token = p["input_1M"] / 1_000_000
    out_per_token = p["output_1M"] / 1_000_000
    return in_per_token, out_per_token
    

In [12]:
# --------- Helpers: normalization --------
_ARTICLES = {"a", "an", "the"}

def strip_punct(s: str) -> str:
    return re.sub(r"[^\w\s&/-]", "", s)

def normalize_item(s: str) -> str:
    s = (s or "").strip().lower()
    s = re.sub(r"\s+", " ", s).strip()
    parts = s.split()
    if parts and parts[0] in _ARTICLES:
        s = " ".join(parts[1:])
    s = strip_punct(s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def split_list_answer(ans: str) -> List[str]:
    ans = (ans or "").strip()
    if not ans:
        return []
    parts = re.split(r"[;\n,]+", ans)
    return [p.strip() for p in parts if p.strip()]

def to_item_list(text: str):
    s = (text or "").strip()
    if not s:
        return []
    # JSON list -> extrae strings entre comillas
    if s.startswith("[") and s.endswith("]"):
        items = re.findall(r'"([^"]+)"', s)
        if items:
            return items
    # fallback ; , \n
    parts = re.split(r"[;\n,]+", s)
    return [p.strip() for p in parts if p.strip()]

def list_metrics_flexible(expected: str, predicted: str):
    exp = {normalize_item(x) for x in to_item_list(expected) if normalize_item(x)}
    pred = {normalize_item(x) for x in to_item_list(predicted) if normalize_item(x)}

    if not exp and not pred:
        return 1.0, 1.0, 1.0, 1
    if not pred:
        return 0.0, 0.0, 0.0, 0

    correct = len(exp & pred)
    precision = correct / max(len(pred), 1)
    recall = correct / max(len(exp), 1)
    f1 = 0.0 if (precision + recall) == 0 else 2 * precision * recall / (precision + recall)
    exact = 1 if exp == pred else 0
    return precision, recall, f1, exact

def exact_match_metric(expected: str, predicted: str):
    return 1 if normalize_item(expected) == normalize_item(predicted) else 0

## Simple Tasks

In [13]:
@dataclass
class RunResult:
    model: str
    task_id: str
    prompt: str
    output: str
    latency_s: float
    input_tokens: int
    output_tokens: int
    #costs (USD)
    cost_input_usd: float = 0.0
    cost_output_usd: float = 0.0
    cost_total_usd: float = 0.0
    # optional eval fields
    precision: Optional[float] = None
    recall: Optional[float] = None
    f1: Optional[float] = None
    exact_match: Optional[int] = None
    correct: Optional[bool] = None

import random

def call_model(model: str, messages: List[Dict[str, str]]) -> RunResult:
    """
    Call Azure model and return text + latency + tokens + estimated USD cost.
    Adds safe retry/backoff for 429 rate limits.
    """
    max_retries = 6          # puedes bajar/subir
    base_sleep_s = 2.0       # base de espera (se hace exponencial)
    
    last_err = None

    for attempt in range(max_retries + 1):
        try:
            t0 = time.time()
            resp = client.chat.completions.create(
                model=model,
                messages=messages
            )
            t1 = time.time()

            choice = resp.choices[0]
            usage = resp.usage

            prompt_tokens = getattr(usage, "prompt_tokens", 0) or 0
            completion_tokens = getattr(usage, "completion_tokens", 0) or 0

            in_per_token, out_per_token = per_token_prices(model)

            cost_input_usd = prompt_tokens * in_per_token
            cost_output_usd = completion_tokens * out_per_token
            cost_total_usd = cost_input_usd + cost_output_usd

            return RunResult(
                model=model,
                task_id="",
                prompt=messages[-1]["content"],
                output=(choice.message.content or "").strip(),
                latency_s=t1 - t0,
                input_tokens=prompt_tokens,
                output_tokens=completion_tokens,
                cost_input_usd=cost_input_usd,
                cost_output_usd=cost_output_usd,
                cost_total_usd=cost_total_usd,
            )

        except Exception as e:
            last_err = e
            msg = str(e)

            # Detecta rate limit (429)
            if "429" in msg or "Too Many Requests" in msg:
                # backoff exponencial + jitter
                sleep_s = base_sleep_s * (2 ** attempt) + random.random()
                print(f"[429] Rate limited on model={model}. Sleeping {sleep_s:.1f}s (attempt {attempt+1}/{max_retries})")
                time.sleep(sleep_s)
                continue

            # otro error distinto -> lo lanzamos
            raise

    # si agotó retries
    raise RuntimeError(f"Failed after retries due to rate limiting. Last error: {last_err}")



In [14]:
SIMPLE_TASKS = [
    {
        "id": "food_1",
        "instruction": "Extract ALL food/drink items mentioned in the text.",
        "input": "The pastor tacos were amazing but the service was slow.", 
        "expected": "pastor tacos",
    },
    {
        "id": "lastname_1",
        "instruction": "Return ONLY the last name(s) from this full name.",
        "input": "Rafael Gallegos Cortés",
        "expected": "Gallegos Cortés",
    },
    {
        "id": "legal_1",
        "instruction": (
            "Identify the names of legal entities in the text and "
            "return them as a list of strings (for example: "
            '["Entity 1", "Entity 2"]).'
        ),
        "input": "The contract is made between Pemex Exploración y Producción, S.A. de C.V. and CFE.",
        "expected": '["Pemex Exploración y Producción, S.A. de C.V.", "CFE"]',
    },
]

In [15]:
def eval_tasks(tasks, models=None, system_prompt=None):
    """
    Evalúa tasks tipo extracción/clasificación.
    
    tasks: pd.DataFrame o list[dict] (como SIMPLE_TASKS)
      columnas esperadas: id, instruction, input, expected
    
    models: lista de deployments. Si None, usa MODELS global.
    system_prompt: si None usa uno por defecto.
    
    return: lista de tuplas (RunResult, expected, instruction, input, task_id)
    """
    if models is None:
        models = MODELS

    if system_prompt is None:
        system_prompt = (
            "You are an information extraction system. "
            "Follow the instruction exactly. "
            "Answer in English. "
            "Do not add explanations."
        )

    # Acepta lista de dicts o DataFrame
    if isinstance(tasks, list):
        tasks_df = pd.DataFrame(tasks)
    else:
        tasks_df = tasks.copy()

    # Validación mínima
    required_cols = {"id", "instruction", "input", "expected"}
    missing = required_cols - set(tasks_df.columns)
    if missing:
        raise ValueError(f"tasks_df is missing columns: {missing}")

    results = []
    for model in models:
        for _, row in tasks_df.iterrows():
            task_id = str(row["id"])
            instruction = str(row["instruction"])
            input_text = str(row["input"])
            expected = "" if pd.isna(row["expected"]) else str(row["expected"])

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{instruction}\n\nText:\n{input_text}"},
            ]

            r = call_model(model, messages) 
            time.sleep(0.2)
            r.task_id = task_id

            results.append((r, expected, instruction, input_text, task_id))

    return results

def summarize_simple(results):
    """
    Summary per model:
      - quality_score (0..1): F1 para list tasks + exact_match para single tasks (promedio)
      - exact_match_rate (global)
      - avg_f1_list_tasks (solo food/legal)
      - avg_latency_s
      - avg_tokens
      - avg_cost_usd, total_cost_usd
      - cost_score (0..1) (mejor = más barato)
      - latency_score (0..1) (mejor = más rápido)
      - overall_score (0..100): combina calidad + costo + latencia
    """
    # Pesos del overall (ajusta si quieres)
    W_QUAL = 0.70
    W_COST = 0.15
    W_LAT  = 0.15

    stats = defaultdict(lambda: {
        "n": 0,
        "exact_match_sum": 0,
        "f1_sum": 0.0,
        "f1_n": 0,
        "quality_sum": 0.0,
        "quality_n": 0,
        "latency_sum": 0.0,
        "input_tokens": 0,
        "output_tokens": 0,
        "cost_total_usd": 0.0,
    })

    # --- 1) Agrega stats + calcula métricas por task ---
    for r, expected, instruction, input_text, task_id in results:
        s = stats[r.model]
        s["n"] += 1
        s["latency_sum"] += r.latency_s
        s["input_tokens"] += r.input_tokens
        s["output_tokens"] += r.output_tokens
        s["cost_total_usd"] += r.cost_total_usd

        is_list_task = str(task_id).startswith("food_") or str(task_id).startswith("legal_")

        if is_list_task:
            p, rc, f1, ex = list_metrics_flexible(expected, r.output)
            r.precision, r.recall, r.f1, r.exact_match = p, rc, f1, ex
            r.correct = bool(ex)

            s["f1_sum"] += f1
            s["f1_n"] += 1
            s["exact_match_sum"] += ex

            task_quality = f1  # calidad para list tasks = F1
        else:
            ex = exact_match_metric(expected, r.output)
            r.exact_match = ex
            r.correct = bool(ex)

            s["exact_match_sum"] += ex
            task_quality = ex  # calidad para single tasks = exact match

        s["quality_sum"] += task_quality
        s["quality_n"] += 1

    # --- 2) Arma DataFrame summary base ---
    summary_rows = []
    for model, s in stats.items():
        n = s["n"] or 1
        exact_match_rate = s["exact_match_sum"] / n
        avg_f1_list_tasks = (s["f1_sum"] / s["f1_n"]) if s["f1_n"] else None
        avg_latency_s = s["latency_sum"] / n
        avg_tokens = (s["input_tokens"] + s["output_tokens"]) / n
        total_cost_usd = s["cost_total_usd"]
        avg_cost_usd = total_cost_usd / n
        quality_score = (s["quality_sum"] / s["quality_n"]) if s["quality_n"] else 0.0

        summary_rows.append({
            "model": model,
            "n": s["n"],
            "quality_score": quality_score,          # 0..1 (promedio de F1/exact)
            "exact_match_rate": exact_match_rate,    # 0..1
            "avg_f1_list_tasks": avg_f1_list_tasks,  # 0..1 o None
            "avg_latency_s": avg_latency_s,
            "avg_tokens": avg_tokens,
            "avg_cost_usd": avg_cost_usd,
            "total_cost_usd": total_cost_usd,
        })

    summary = pd.DataFrame(summary_rows)

    # --- 3) Normaliza costo y latencia a 0..1 (1 = mejor) y calcula overall ---
    def minmax_better_low(x, xmin, xmax):
        # si todos iguales, score neutro
        if xmax == xmin:
            return 1.0
        return (xmax - x) / (xmax - xmin)

    cost_min, cost_max = summary["avg_cost_usd"].min(), summary["avg_cost_usd"].max()
    lat_min, lat_max = summary["avg_latency_s"].min(), summary["avg_latency_s"].max()

    summary["cost_score"] = summary["avg_cost_usd"].apply(lambda x: minmax_better_low(x, cost_min, cost_max))
    summary["latency_score"] = summary["avg_latency_s"].apply(lambda x: minmax_better_low(x, lat_min, lat_max))

    summary["overall_score"] = 100 * (
        W_QUAL * summary["quality_score"] +
        W_COST * summary["cost_score"] +
        W_LAT  * summary["latency_score"]
    )

    return summary.sort_values(["overall_score", "avg_cost_usd"], ascending=[False, True])

In [16]:
results_small = eval_tasks(SIMPLE_TASKS)


In [17]:
summary_small = summarize_simple(results_small)
summary_small

summary_small.style.format({
    "avg_latency_s": "{:.3f}",
    "avg_tokens": "{:.1f}",
    "total_cost_usd": "{:.2e}",
})  

,model,n,quality_score,exact_match_rate,avg_f1_list_tasks,avg_latency_s,avg_tokens,avg_cost_usd,total_cost_usd,cost_score,latency_score,overall_score
0,gpt-5-chat,3,1.000000,1.000000,1.000000,0.533,76.7,0.000004,1.24e-05,0.936489,1.000000,99.047334
4,DeepSeek-V3.1,3,1.000000,1.000000,1.000000,0.763,74.3,0.000003,9.25e-06,1.000000,0.934172,99.012574
2,gpt-5-nano,3,1.000000,1.000000,1.000000,3.528,447.3,0.000004,1.32e-05,0.920631,0.141253,85.928268
3,o4-mini,3,1.000000,1.000000,1.000000,2.612,243.0,0.000010,2.92e-05,0.602357,0.403802,85.092380
1,gpt-5.2-chat,3,1.000000,1.000000,1.000000,4.020,255.3,0.000020,5.93e-05,0.000000,0.000000,70.000000


In [18]:
rows = []
for r, expected, instruction, input_text, task_id in results_small:
    rows.append({
        "model": r.model,
        "task_id": task_id,
        "instruction": instruction,
        "input_text": input_text,
        "expected": expected,
        "output": r.output,
        "correct": r.correct,
        "exact_match": r.exact_match,
        "precision": r.precision,
        "recall": r.recall,
        "f1": r.f1,
        "latency_s": round(r.latency_s, 3),
        "input_tokens": r.input_tokens,
        "output_tokens": r.output_tokens,
        "cost_input_usd": r.cost_input_usd,
        "cost_output_usd": r.cost_output_usd,
        "cost_total_usd": r.cost_total_usd,
    })

df_results_simple = pd.DataFrame(rows)
df_results_simple.head().style.format({
    "latency_s": "{:.3f}",
    "cost_input_usd": "{:.2e}",
    "cost_output_usd": "{:.2e}",
    "cost_total_usd": "{:.2e}",
    "precision": "{:.1f}",
    "recall": "{:.1f}",
    "f1": "{:.1f}",
})


,model,task_id,instruction,input_text,expected,output,correct,exact_match,precision,recall,f1,latency_s,input_tokens,output_tokens,cost_input_usd,cost_output_usd,cost_total_usd
0,gpt-5-chat,food_1,Extract ALL food/drink items mentioned in the text.,The pastor tacos were amazing but the service was slow.,pastor tacos,pastor tacos,True,1,1.0,1.0,1.0,0.500,56,4,2.80e-06,3.20e-07,3.12e-06
1,gpt-5-chat,lastname_1,Return ONLY the last name(s) from this full name.,Rafael Gallegos Cortés,Gallegos Cortés,Gallegos Cortés,True,1,nan,nan,nan,0.340,53,6,2.65e-06,4.80e-07,3.13e-06
2,gpt-5-chat,legal_1,"Identify the names of legal entities in the text and return them as a list of strings (for example: [""Entity 1"", ""Entity 2""]).","The contract is made between Pemex Exploración y Producción, S.A. de C.V. and CFE.","[""Pemex Exploración y Producción, S.A. de C.V."", ""CFE""]","[""Pemex Exploración y Producción, S.A. de C.V."", ""CFE""]",True,1,1.0,1.0,1.0,0.759,90,21,4.50e-06,1.68e-06,6.18e-06
3,gpt-5.2-chat,food_1,Extract ALL food/drink items mentioned in the text.,The pastor tacos were amazing but the service was slow.,pastor tacos,pastor tacos,True,1,1.0,1.0,1.0,6.012,55,269,3.85e-06,2.15e-05,2.54e-05
4,gpt-5.2-chat,lastname_1,Return ONLY the last name(s) from this full name.,Rafael Gallegos Cortés,Gallegos Cortés,Gallegos Cortés,True,1,nan,nan,nan,1.946,52,79,3.64e-06,6.32e-06,9.96e-06


In [19]:
for _, row in df_results_simple.iterrows():
    print("="*60)
    print(f"Model:   {row['model']}")
    print(f"Task:    {row['task_id']}")
    print(f"Instruction:\n{row['instruction']}")
    print(f"\nInput text:\n{row['input_text']}")
    print(f"\nExpected: {row['expected']}")
    print(f"Output:   {row['output']}")
    print(f"Correct:  {row['correct']}")
    print(f"Latency:  {row['latency_s']} s | tokens in/out: {row['input_tokens']}/{row['output_tokens']}")
    print(f"Cost USD - Input: {row['cost_input_usd']:.2e}, Output: {row['cost_output_usd']:.2e}, Total: {row['cost_total_usd']:.2e}\n")

Model:   gpt-5-chat
Task:    food_1
Instruction:
Extract ALL food/drink items mentioned in the text.

Input text:
The pastor tacos were amazing but the service was slow.

Expected: pastor tacos
Output:   pastor tacos
Correct:  True
Latency:  0.5 s | tokens in/out: 56/4
Cost USD - Input: 2.80e-06, Output: 3.20e-07, Total: 3.12e-06

Model:   gpt-5-chat
Task:    lastname_1
Instruction:
Return ONLY the last name(s) from this full name.

Input text:
Rafael Gallegos Cortés

Expected: Gallegos Cortés
Output:   Gallegos Cortés
Correct:  True
Latency:  0.34 s | tokens in/out: 53/6
Cost USD - Input: 2.65e-06, Output: 4.80e-07, Total: 3.13e-06

Model:   gpt-5-chat
Task:    legal_1
Instruction:
Identify the names of legal entities in the text and return them as a list of strings (for example: ["Entity 1", "Entity 2"]).

Input text:
The contract is made between Pemex Exploración y Producción, S.A. de C.V. and CFE.

Expected: ["Pemex Exploración y Producción, S.A. de C.V.", "CFE"]
Output:   ["Pemex 

In [20]:
summary_small.sort_values(["overall_score","total_cost_usd"], ascending=[False, True])


,model,n,quality_score,exact_match_rate,avg_f1_list_tasks,avg_latency_s,avg_tokens,avg_cost_usd,total_cost_usd,cost_score,latency_score,overall_score
0,gpt-5-chat,3,1.0,1.0,1.0,0.533079,76.666667,0.000004,0.000012,0.936489,1.000000,99.047334
4,DeepSeek-V3.1,3,1.0,1.0,1.0,0.762634,74.333333,0.000003,0.000009,1.000000,0.934172,99.012574
2,gpt-5-nano,3,1.0,1.0,1.0,3.527685,447.333333,0.000004,0.000013,0.920631,0.141253,85.928268
3,o4-mini,3,1.0,1.0,1.0,2.612130,243.000000,0.000010,0.000029,0.602357,0.403802,85.092380
1,gpt-5.2-chat,3,1.0,1.0,1.0,4.020261,255.333333,0.000020,0.000059,0.000000,0.000000,70.000000


### Additional tasks

In [21]:
#Adddiotal tasks
TASK_INSTRUCTIONS = {
    "extract_food_items":
        "Extract food/drink items referred into the text. "
        "Do NOT infer, filter, or judge; just extract the item that are mentioned. "
        "Return an empty string if none.",
    "extract_last_name":
        "Extract the last name(s) from the full name. "
        "Return ONLY the last name(s) as they appear (including particles like 'de', 'del', 'bin' when present). "
        "Return a single string.",
    "extract_legal_entities":
        "Extract the legal entity names mentioned in the text. "
        "Preserve the entity names as written. "
        "Return a semicolon-separated list. "
        "If none, return an empty string.",
    "extract_decision":
        "Extract the final decision from the text. "
        "Return ONLY one of: approved, denied, pending. "
        "If unclear, choose pending.",
    "extract_primary_date_iso":
        "Extract the PRIMARY date mentioned (the first main date / the main deadline / the main event date). "
        "Return ONLY the date in ISO format YYYY-MM-DD. "
        "If no date, return an empty string.",
    "detect_contradiction":
        "Does the text contain a contradiction? "
        "Return ONLY one of: yes, no, maybe. "
        "Use 'maybe' only if it's genuinely ambiguous.",
}

In [22]:
results_csv = eval_tasks(tasks_df, system_prompt=SYSTEM_SIMPLE)

[429] Rate limited on model=DeepSeek-V3.1. Sleeping 2.9s (attempt 1/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 4.9s (attempt 2/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 8.3s (attempt 3/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 16.3s (attempt 4/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 33.0s (attempt 5/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 2.7s (attempt 1/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 4.4s (attempt 2/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 8.2s (attempt 3/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 16.5s (attempt 4/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 2.2s (attempt 1/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 2.4s (attempt 1/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 2.7s (attempt 1/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 4.2s (attempt 2/6)
[429] Rate limited on model=DeepSeek-V3.1. Sleeping 8.1s (attempt 3/6)
[42

In [23]:
summary_csv = summarize_simple(results_csv)
summary_csv

,model,n,quality_score,exact_match_rate,avg_f1_list_tasks,avg_latency_s,avg_tokens,avg_cost_usd,total_cost_usd,cost_score,latency_score,overall_score
4,DeepSeek-V3.1,66,0.924242,0.924242,1.000000,0.606660,90.893939,0.000004,0.000244,0.944544,1.000000,93.865131
0,gpt-5-chat,66,0.918182,0.893939,0.983333,0.867347,91.333333,0.000005,0.000313,0.842416,0.878468,90.085984
2,gpt-5-nano,66,0.943434,0.909091,0.969444,2.374926,321.409091,0.000003,0.000207,1.000000,0.175634,83.674913
3,o4-mini,66,0.913131,0.878788,0.969444,2.014066,209.303030,0.000008,0.000553,0.489220,0.343867,76.415494
1,gpt-5.2-chat,66,0.882828,0.848485,0.969444,2.751661,177.954545,0.000013,0.000884,0.000000,0.000000,61.797980


In [24]:
rows = []
for r, expected, instruction, input_text, task_id in results_csv:
    rows.append({
        "model": r.model,
        "task_id": task_id,
        "instruction": instruction,
        "input_text": input_text,
        "expected": expected,
        "output": r.output,
        "correct": r.correct,
        "exact_match": r.exact_match,
        "recall": r.recall,
        "f1": r.f1,
        "latency_s": round(r.latency_s, 3),
        "input_tokens": r.input_tokens,
        "output_tokens": r.output_tokens,
        "cost_input_usd": r.cost_input_usd,
        "cost_output_usd": r.cost_output_usd,
        "cost_total_usd": r.cost_total_usd,
    })

df_results = pd.DataFrame(rows)
df_results.head()


,model,task_id,instruction,input_text,expected,output,correct,exact_match,recall,f1,latency_s,input_tokens,output_tokens,cost_input_usd,cost_output_usd,cost_total_usd
0,gpt-5-chat,food_001,Extract ALL food/drink items mentioned in the ...,"The food was not good, but I liked how the wai...",hamburger,hamburger;hamburger,True,1,1.0,1.0,0.454,108,6,0.000005,4.800000e-07,0.000006
1,gpt-5-chat,food_002,Extract ALL food/drink items mentioned in the ...,"I came for coffee, stayed for the croissant. T...",coffee; croissant,coffee;croissant;croissant,True,1,1.0,1.0,0.333,103,8,0.000005,6.400000e-07,0.000006
2,gpt-5-chat,food_003,Extract ALL food/drink items mentioned in the ...,We didn't order dessert. The pizza was great a...,pizza; salad,dessert;pizza;salad,False,0,1.0,0.8,0.378,95,8,0.000005,6.400000e-07,0.000005
3,gpt-5-chat,food_004,Extract ALL food/drink items mentioned in the ...,"The sushi was overpriced; however, the miso so...",sushi; miso soup,sushi; miso soup,True,1,1.0,1.0,0.386,94,7,0.000005,5.600000e-07,0.000005
4,gpt-5-chat,food_005,Extract ALL food/drink items mentioned in the ...,"I asked for fries, but they brought onion ring...",fries; onion rings; burger,fries;onion rings;burger,True,1,1.0,1.0,0.483,97,9,0.000005,7.200000e-07,0.000006


In [25]:
for _, row in df_results.iterrows():
    print("="*60)
    print(f"Model:   {row['model']}")
    print(f"Task:    {row['task_id']}")
    print(f"Instruction:\n{row['instruction']}")
    print(f"\nInput text:\n{row['input_text']}")
    print(f"\nExpected: {row['expected']}")
    print(f"Output:   {row['output']}")
    print(f"Correct:  {row['correct']}")
    print(f"Latency:  {row['latency_s']} s | tokens in/out: {row['input_tokens']}/{row['output_tokens']}")
    print(f"Cost USD - Input: {row['cost_input_usd']:.2e}, Output: {row['cost_output_usd']:.2e}, Total: {row['cost_total_usd']:.2e}\n")


Model:   gpt-5-chat
Task:    food_001
Instruction:
Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.

Input text:
The food was not good, but I liked how the waitress apologized. I liked the hamburger, but the hamburger was not from this place.

Expected: hamburger
Output:   hamburger;hamburger
Correct:  True
Latency:  0.454 s | tokens in/out: 108/6
Cost USD - Input: 5.40e-06, Output: 4.80e-07, Total: 5.88e-06

Model:   gpt-5-chat
Task:    food_002
Instruction:
Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.

Input text:
I came for coffee, stayed for the croissant. The service was slow, but the croissant was perfect.

Expected: coffee; croissant
Output:   coffe

## Analytical tasks

In [26]:
# @dataclass
# class RunResult:
#     model: str
#     task_id: str
#     prompt: str
#     output: str
#     latency_s: float
#     input_tokens: int
#     output_tokens: int
#     #Costs (USD)
#     cost_input_usd: float = 0.0
#     cost_output_usd: float = 0.0
#     cost_total_usd: float = 0.0

# def call_model(model: str, messages: List[Dict[str, str]], max_tokens: int | None = None) -> RunResult:
#     """
#     Llama al modelo de Azure y regresa un objeto con el texto + métricas básicas.
#     Para gpt-5-chat usamos 'max_completion_tokens' en lugar de 'max_tokens'.
#     """
#     t0 = time.time()

#     kwargs = {}
#     if max_tokens is not None:
#         # Azure GPT-5 usa este nombre de parámetro:
#         kwargs["max_completion_tokens"] = max_tokens

#     resp = client.chat.completions.create(
#         model=model,
#         messages=messages,
#         **kwargs,
#     )
#     t1 = time.time()

#     choice = resp.choices[0]
#     usage = resp.usage

#     return RunResult(
#         model=model,
#         task_id="",
#         prompt=messages[-1]["content"],
#         output=choice.message.content,
#         latency_s=t1 - t0,
#         input_tokens=usage.prompt_tokens,
#         output_tokens=usage.completion_tokens,
#     )

In [27]:
ANALYTICAL_TASKS = [
    {
        "id": "missing_1",
        "description": "Missing values analysis",
        "prompt": """
You are a senior data analyst. I have this dataset summary (in CSV):

column,missing_count,mean,std,min,max
age,10,35,7,18,65
income,50,25000,12000,5000,90000
city,0,NA,NA,NA,NA

1) Describe the missing value pattern.
2) Propose a reasonable imputation strategy.
3) Mention risks or caveats.
"""
    },
    {
        "id": "ts_1",
        "description": "Time series analysis",
        "prompt": """
You are a time series expert. I have this monthly revenue series (index, month, value):

1,2023-01,100
2,2023-02,110
3,2023-03,130
4,2023-04,140
5,2023-05,160
6,2023-06,170
7,2023-07,200
8,2023-08,210
9,2023-09,180
10,2023-10,220
11,2023-11,230
12,2023-12,250

1) Describe trend and any anomalies.
2) Give a short qualitative forecast for the next 3 months.
"""
    },
    {
        "id": "stats_1",
        "description": "Statistical insight on multivariate data",
        "prompt": """
You are a senior data scientist. I have this dataset summary:

- 1000 rows.
- Variables:
    * age (numeric, 18-80, mean=40, std=12)
    * income (numeric, strongly right-skewed, many values close to 0, some very large)
    * churn (binary: 1 if customer left, 0 otherwise)
    * segment (categorical: A, B, C)

Tasks:
1) Propose 2-3 hypotheses that would be interesting to test statistically.
2) Explain briefly which statistical methods you would use for each hypothesis.
3) Suggest 2 simple visualizations to better understand the data.
"""
    },
]


In [28]:
def run_analytical_tasks():
    rows = []
    for model in MODELS:
        for task in ANALYTICAL_TASKS:
            messages = [
                {"role": "system", "content": "You are an expert data analyst."},
                {"role": "user", "content": task["prompt"]},
            ]
            # r = call_model(model, messages, max_tokens=600) No funciona el maximo para algunos modelos
            r = call_model(model, messages)
            r.task_id = task["id"]
            rows.append(r)
    return rows

results_analytical = run_analytical_tasks()
len(results_analytical)


15

In [29]:
df_analytical = pd.DataFrame([r.__dict__ for r in results_analytical])
df_analytical

,model,task_id,prompt,output,latency_s,input_tokens,output_tokens,cost_input_usd,cost_output_usd,cost_total_usd,precision,recall,f1,exact_match,correct
0,gpt-5-chat,missing_1,\nYou are a senior data analyst. I have this d...,Let's analyze this step by step.\n\n---\n\n###...,8.585562,111,631,5.550000e-06,0.000050,0.000056,None,None,None,None,None
1,gpt-5-chat,ts_1,\nYou are a time series expert. I have this mo...,Let's analyze your monthly revenue time series...,4.881167,169,452,8.450000e-06,0.000036,0.000045,None,None,None,None,None
2,gpt-5-chat,stats_1,\nYou are a senior data scientist. I have this...,Sure — here’s how you might proceed as a senio...,4.686767,159,525,7.950000e-06,0.000042,0.000050,None,None,None,None,None
3,gpt-5.2-chat,missing_1,\nYou are a senior data analyst. I have this d...,Here’s a structured analysis based on the summ...,9.208923,110,565,7.700000e-06,0.000045,0.000053,None,None,None,None,None
4,gpt-5.2-chat,ts_1,\nYou are a time series expert. I have this mo...,Here’s a concise time‑series interpretation.\n...,6.470315,168,348,1.176000e-05,0.000028,0.000040,None,None,None,None,None
5,gpt-5.2-chat,stats_1,\nYou are a senior data scientist. I have this...,Here’s a structured way to approach this datas...,8.314432,158,680,1.106000e-05,0.000054,0.000065,None,None,None,None,None
6,gpt-5-nano,missing_1,\nYou are a senior data analyst. I have this d...,"Here is a concise read on the pattern, a pract...",29.110495,110,4294,9.900000e-07,0.000043,0.000044,None,None,None,None,None
7,gpt-5-nano,ts_1,\nYou are a time series expert. I have this mo...,Here’s a concise read of the series and a shor...,27.724897,168,3667,1.512000e-06,0.000037,0.000038,None,None,None,None,None
8,gpt-5-nano,stats_1,\nYou are a senior data scientist. I have this...,"Here are 3 concise, testable ideas using the g...",19.863030,158,2764,1.422000e-06,0.000028,0.000029,None,None,None,None,None
9,o4-mini,missing_1,\nYou are a senior data analyst. I have this d...,1) Missing-value pattern \n- Two of the three...,7.401806,110,1127,4.400000e-06,0.000045,0.000049,None,None,None,None,None


In [30]:
# Save analytical results to CSV
df_analytical.to_excel("analytical_results.xlsx", index=False)


### Statistics and Missing values from Mimic and Titanic

In [31]:
# Mimic
# train = pd.read_csv('mimic_train.csv')
# train.head(2)

# --- MIMIC: ICU mortality dataset ---
mimic_df = pd.read_csv("mimic_train.csv")

# small sample
mimic_sample = mimic_df.head(100).to_string(index=False)

# describe only numeric (to avoid too much output)
mimic_summary = mimic_df.describe(include="number").to_string()

# missing value count
mimic_missing_df = mimic_df.isna().sum().reset_index()
mimic_missing_df.columns = ["column", "missing_count"]
mimic_missing = mimic_missing_df.to_string(index=False)

# basic target distribution
if "HOSPITAL_EXPIRE_FLAG" in mimic_df.columns:
    mimic_target_dist = mimic_df["HOSPITAL_EXPIRE_FLAG"].value_counts(normalize=True).to_frame("proportion").to_string()
else:
    mimic_target_dist = "Column HOSPITAL_EXPIRE_FLAG not found."


# --- TITANIC: classic survival dataset ---
titanic_df = pd.read_csv("titanic3.csv")

titanic_sample = titanic_df.head(100).to_string(index=False)
titanic_summary = titanic_df.describe(include="number").to_string()

titanic_missing_df = titanic_df.isna().sum().reset_index()
titanic_missing_df.columns = ["column", "missing_count"]
titanic_missing = titanic_missing_df.to_string(index=False)

# survival distribution if exists
if "survived" in titanic_df.columns:
    titanic_target_dist = titanic_df["survived"].value_counts(normalize=True).to_frame("proportion").to_string()
else:
    titanic_target_dist = "Column survived not found."


### Mimic Task

In [32]:
ANALYTICAL_TASKS = [

###################################################################################################################################
#############################TAREA 1. Missing values and EDA in MIMIC dataset
    {
        "id": "mimic_eda_missing",
        "description": "EDA and missing values in ICU mortality dataset (MIMIC)",
        "prompt": f"""
You are a senior data analyst working with ICU data (MIMIC-III style).
We want to predict in-hospital mortality for ICU patients (HOSPITAL_EXPIRE_FLAG).

Here is the sample dataset:

{mimic_sample}

Here is the count of missing values per column:

{mimic_missing}

Here is a summary of basic statistics for numeric variables:

{mimic_summary}

Here is the distribution of the target HOSPITAL_EXPIRE_FLAG (proportion):

{mimic_target_dist}

Tasks:
1) Describe the overall structure of the dataset (types of variables, what they seem to represent).
2) Analyze the missing value pattern: which variables are more problematic and what might be the underlying reasons in an ICU context.
3) Propose concrete strategies to handle missing values (e.g., dropping rows/columns, different imputations) and justify them.
4) Mention at least 3 potential risks or caveats, especially regarding data leakage and bias in a medical setting.
"""
    },

##################################################################################################################################
##################################Task 2. Modeling design for MIMIC dataset

{
        "id": "mimic_modeling",
        "description": "Model design for ICU mortality prediction (MIMIC)",
        "prompt": f"""
You are a senior data scientist helping to build a model that predicts in-hospital mortality
for ICU patients using the variable HOSPITAL_EXPIRE_FLAG as the target.

You have access to the same dataset as before, with vitals, demographics, and other ICU-related variables.

You know the following:
- The dataset has missing values as shown below (counts per column):

{mimic_missing}

- The numeric variables have the following basic statistics:

{mimic_summary}

- The target distribution (HOSPITAL_EXPIRE_FLAG) is:

{mimic_target_dist}

Tasks:
1) Propose a reasonable baseline modeling approach (e.g., logistic regression, tree-based model, etc.) and explain why.
2) Describe how you would preprocess the data: handling missing values, scaling, encoding categorical variables, and dealing with highly correlated features.
3) Discuss how you would handle potential class imbalance in HOSPITAL_EXPIRE_FLAG.
4) Suggest appropriate evaluation metrics for this medical prediction problem and explain why they are suitable (e.g., ROC-AUC, PR-AUC, calibration).
5) Mention at least 2 ways to check whether the model might be unfair or biased toward some patient subgroups.
"""
    }
]


In [33]:
def run_analytical_tasks():
    rows = []
    for model in MODELS:
        for task in ANALYTICAL_TASKS:
            messages = [
                {"role": "system", "content": "You are an expert data analyst."},
                {"role": "user", "content": task["prompt"]},
            ]
            # r = call_model(model, messages, max_tokens=800) No funciona el maximo para algunos modelos
            r = call_model(model, messages)
            r.task_id = task["id"]
            rows.append(r)
    return rows

results_analytical = run_analytical_tasks()
len(results_analytical)


10

In [34]:
df_analytical = pd.DataFrame([r.__dict__ for r in results_analytical])
df_analytical.head()

,model,task_id,prompt,output,latency_s,input_tokens,output_tokens,cost_input_usd,cost_output_usd,cost_total_usd,precision,recall,f1,exact_match,correct
0,gpt-5-chat,mimic_eda_missing,\nYou are a senior data analyst working with I...,Let's go step‑by‑step carefully and thoroughly...,27.490518,23630,1905,0.001182,0.000152,0.001334,None,None,None,None,None
1,gpt-5-chat,mimic_modeling,\nYou are a senior data scientist helping to b...,Let's go step-by-step carefully and systematic...,19.369486,2278,1420,0.000114,0.000114,0.000228,None,None,None,None,None
2,gpt-5.2-chat,mimic_eda_missing,\nYou are a senior data analyst working with I...,"Below is a structured, ICU‑aware analysis of y...",28.113982,23629,1805,0.001654,0.000144,0.001798,None,None,None,None,None
3,gpt-5.2-chat,mimic_modeling,\nYou are a senior data scientist helping to b...,"Below is a practical, ICU‑appropriate modeling...",21.668944,2277,1302,0.000159,0.000104,0.000264,None,None,None,None,None
4,gpt-5-nano,mimic_eda_missing,\nYou are a senior data analyst working with I...,"Here is a concise, actionable read on the data...",39.958165,23629,6406,0.000213,0.000064,0.000277,None,None,None,None,None


In [35]:
## Mimic
df_analytical.to_excel("analytical_results_mimic.xlsx", index=False)


### Titanic Task

In [36]:
ANALYTICAL_TASKS = [

##################################################################################################################################
##################################TAREA 1. Missing values and EDA in Titanic dataset

{
        "id": "titanic_missing",
        "description": "Missing values and basic EDA in Titanic survival dataset",
        "prompt": f"""
You are a data analyst working with the Titanic passenger dataset.

Here is a sample of the data (first 5 rows):

{titanic_sample}

Here is the count of missing values per column:

{titanic_missing}

Here is a summary of basic statistics for numeric variables:

{titanic_summary}

Here is the distribution of the target 'survived' (proportion):

{titanic_target_dist}

Tasks:
1) Identify the most relevant variables with missing values (e.g., age, cabin, embarked) and describe how they might affect the analysis.
2) Propose at least two different imputation strategies for the 'age' variable and discuss pros and cons of each.
3) Propose a reasonable way to handle the 'cabin' variable given that it has many missing values and a large number of categories.
4) Suggest 2–3 visualizations that would help understand the relationship between survival and key variables (e.g., sex, class, age).
"""
    },



##################################################################################################################################
##################################Task 2. Modeling design for Titanic dataset

    {
        "id": "titanic_modeling",
        "description": "Model design for predicting survival on Titanic dataset",
        "prompt": f"""
You are a senior data scientist helping to build a model that predicts passenger survival
on the Titanic dataset (target variable: 'survived').

You have the following information:

- Sample of the dataset:

{titanic_sample}

- Numeric summary:

{titanic_summary}

- Missing value counts:

{titanic_missing}

- Target distribution:

{titanic_target_dist}

Tasks:
1) Propose a baseline modeling approach (e.g., logistic regression) and one more flexible model (e.g., tree-based model) and explain the intuition behind both.
2) Explain how you would preprocess the data: encoding categorical variables (e.g., sex, class), handling missing values, and possibly engineering new features.
3) Suggest which evaluation metrics you would use and why (e.g., accuracy, F1-score, ROC-AUC).
4) Mention 2–3 model diagnostics or validation strategies you would apply (e.g., cross-validation, learning curves, calibration plots).
"""
    },
]


In [37]:
results_analytical = run_analytical_tasks()
len(results_analytical)

10

In [38]:
df_analytical = pd.DataFrame([r.__dict__ for r in results_analytical])
df_analytical.to_csv("analytical_results_titanic.csv", index=False)
df_analytical.head()


,model,task_id,prompt,output,latency_s,input_tokens,output_tokens,cost_input_usd,cost_output_usd,cost_total_usd,precision,recall,f1,exact_match,correct
0,gpt-5-chat,titanic_missing,\nYou are a data analyst working with the Tita...,Let’s address each task systematically as a da...,19.711553,6256,1399,0.000313,0.000112,0.000425,None,None,None,None,None
1,gpt-5-chat,titanic_modeling,\nYou are a senior data scientist helping to b...,Let's go step-by-step: \n\n---\n\n## 1️⃣ Base...,17.412826,6255,1241,0.000313,0.000099,0.000412,None,None,None,None,None
2,gpt-5.2-chat,titanic_missing,\nYou are a data analyst working with the Tita...,Below is a structured analytical response to t...,18.844210,6255,1241,0.000438,0.000099,0.000537,None,None,None,None,None
3,gpt-5.2-chat,titanic_modeling,\nYou are a senior data scientist helping to b...,Below is how I would approach this as a senior...,17.474355,6254,1126,0.000438,0.000090,0.000528,None,None,None,None,None
4,gpt-5-nano,titanic_missing,\nYou are a data analyst working with the Tita...,Here are practical recommendations tailored to...,36.197774,6255,5974,0.000056,0.000060,0.000116,None,None,None,None,None


## Summary

In [39]:
summary = (
    df_analytical.groupby("model")
    .agg(
        n_calls=("task_id", "count"),
        total_input_tokens=("input_tokens", "sum"),
        total_output_tokens=("output_tokens", "sum"),
        total_cost_usd=("cost_total_usd", "sum"),
        avg_latency_s=("latency_s", "mean"),
    )
    .reset_index()
    .sort_values("total_cost_usd")
)

summary


,model,n_calls,total_input_tokens,total_output_tokens,total_cost_usd,avg_latency_s
2,gpt-5-nano,2,12509,10155,0.000214,31.503919
4,o4-mini,2,12509,2204,0.000589,10.019533
0,DeepSeek-V3.1,2,10813,3204,0.000593,17.422102
1,gpt-5-chat,2,12511,2640,0.000837,18.562190
3,gpt-5.2-chat,2,12509,2367,0.001065,18.159282


In [40]:
df_results.style.format({
    "cost_input_usd": "{:.3e}",
    "cost_output_usd": "{:.3e}",
    "cost_total_usd": "{:.3e}",
})

,model,task_id,instruction,input_text,expected,output,correct,exact_match,recall,f1,latency_s,input_tokens,output_tokens,cost_input_usd,cost_output_usd,cost_total_usd
0,gpt-5-chat,food_001,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.","The food was not good, but I liked how the waitress apologized. I liked the hamburger, but the hamburger was not from this place.",hamburger,hamburger;hamburger,True,1,1.000000,1.000000,0.454000,108,6,5.400e-06,4.800e-07,5.880e-06
1,gpt-5-chat,food_002,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.","I came for coffee, stayed for the croissant. The service was slow, but the croissant was perfect.",coffee; croissant,coffee;croissant;croissant,True,1,1.000000,1.000000,0.333000,103,8,5.150e-06,6.400e-07,5.790e-06
2,gpt-5-chat,food_003,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.",We didn't order dessert. The pizza was great and the salad was fresh.,pizza; salad,dessert;pizza;salad,False,0,1.000000,0.800000,0.378000,95,8,4.750e-06,6.400e-07,5.390e-06
3,gpt-5-chat,food_004,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.","The sushi was overpriced; however, the miso soup was comforting.",sushi; miso soup,sushi; miso soup,True,1,1.000000,1.000000,0.386000,94,7,4.700e-06,5.600e-07,5.260e-06
4,gpt-5-chat,food_005,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.","I asked for fries, but they brought onion rings instead. The burger was fine.",fries; onion rings; burger,fries;onion rings;burger,True,1,1.000000,1.000000,0.483000,97,9,4.850e-06,7.200e-07,5.570e-06
5,gpt-5-chat,food_006,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.",No tacos today—just a burrito and a soda.,burrito; soda,tacos; burrito; soda,False,0,1.000000,0.800000,0.453000,92,8,4.600e-06,6.400e-07,5.240e-06
6,gpt-5-chat,food_007,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.",The steak wasn't cooked right. The mashed potatoes were excellent though.,steak; mashed potatoes,steak;mashed potatoes,True,1,1.000000,1.000000,0.439000,93,7,4.650e-06,5.600e-07,5.210e-06
7,gpt-5-chat,food_008,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.",Loved the ice cream! I didn't try the cake.,ice cream; cake,ice cream;cake,True,1,1.000000,1.000000,0.398000,91,5,4.550e-06,4.000e-07,4.950e-06
8,gpt-5-chat,food_009,"Extract ALL food/drink items mentioned in the text. Do NOT infer, filter, or judge; just extract items that are mentioned. Return a semicolon-separated list in the order they appear. Return an empty string if none.","The menu mentioned ramen, but I ordered udon.",ramen; udon,ramen;udon,True,1,1.000000,1.000000,0.812000,91,6,4.550e-06,4.800e-07,5.030e-06
9,gpt-5-chat,food_010,"Extract ALL food/drink items mentioned in the tex

In [41]:
summary = summarize_simple(results_csv)
summary["overall_score"] = summary["exact_match_rate"].fillna(0)  # o combinar con f1 si quieres

best = summary.sort_values(["overall_score","total_cost_usd"], ascending=[False, True]).iloc[0]
best


model                DeepSeek-V3.1
n                               66
quality_score             0.924242
exact_match_rate          0.924242
avg_f1_list_tasks              1.0
avg_latency_s              0.60666
avg_tokens               90.893939
avg_cost_usd              0.000004
total_cost_usd            0.000244
cost_score                0.944544
latency_score                  1.0
overall_score             0.924242
Name: 4, dtype: object

# Chatbot

##  Chat from input linea

In [42]:
def mini_chat(model: str = "gpt-5-chat"):
    """
    Mini chatbot in console using your Azure deployment.
    Keeps the conversation history.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert data analysis, statistics, missing values and time series."
            ),
        }
    ]

    print("=== Mini chatbot with Azure (model: {}) ===".format(model))
    print("Write 'salir', 'exit' or 'quit' to end the conversation.\n")

    while True:
        user = input("You: ")
        if user.strip().lower() in ("salir", "exit", "quit"):
            print("Bot: ¡Hasta luego, Good Bye! 👋")
            break

        messages.append({"role": "user", "content": user})

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            #max_completion_tokens=300,  # O ajusta si quieres respuestas más cortas/largas
        )

        answer = response.choices[0].message.content
        print(f"\nBot: {answer}\n")

        messages.append({"role": "assistant", "content": answer})


In [43]:
mini_chat("gpt-5-chat")


=== Mini chatbot with Azure (model: gpt-5-chat) ===
Write 'salir', 'exit' or 'quit' to end the conversation.


Bot: Hi there! 👋  
How can I help you today? Are you working on some data analysis or time series problem, or do you have a question about handling missing values?


Bot: Hello! 👋  
Looks like your message came through empty — could you please resend your question or describe what you’d like help with?  
I can assist with data cleaning, time series forecasting, missing data imputation, and statistical analysis.

Bot: ¡Hasta luego, Good Bye! 👋


## Chatbot in the terminal

Run the script **"chat_terminal.py"** in the terminal from the excelexorcist project folder with the next command:

"python chat_terminal.py"

## Chatbot in streamlit

In [44]:
#!pip install streamlit openai python-dotenv

Run the script **"app.py"** in terminal from excelexorcist project folder with the next command:


streamlit run app.py